In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(1, '../../')

In [14]:
import torch
from ecg_embed import EcgEmbed

batch = torch.rand(64, 12, 5000)
embed = EcgEmbed()([batch, None])
print(embed[0].shape)

torch.Size([64, 240])
